In [2]:
# for loading json files
import json

# for easy math 
import numpy as np

# for loading the directory of the file
from os import listdir

# for creating models
import gensim
from gensim.models import doc2vec

# to work with pandas
import pandas as pd

In [3]:
# # docLabels are the names of the documents, which could be the keys for classification
# docLabels = []
# docLabels = [file for file in listdir("data") ]

In [4]:
# docLabels

In [5]:
# json_documents = {}
# for i in docLabels:
#     json_documents[i]=json.load(open('data/'+i))

In [6]:
# json_documents['squad_dev_doc.json'][0]

In [7]:
# json_documents[1][0]

In [11]:
# Getting the json files into a dataframe
df_squad_train = pd.read_json('squad_train_doc.json')
# df_squad_dev = pd.read_json('data/squad_dev_doc.json')
# df_dev = pd.read_json('data/dev-v1.1.json')
# df_train = pd.read_json('data/train-v1.1.json')

In [12]:
# Renaming the columns for the purpose of understanding
df_squad_train.rename(columns={'passages':'documents'}, inplace=True)

In [13]:
df_squad_train.head()

,documents,title
0,"[{'context': 'Architecturally, the school has ...",University_of_Notre_Dame
1,[{'context': 'Beyoncé Giselle Knowles-Carter (...,Beyoncé
2,[{'context': 'Montana i/mɒnˈtænə/ is a state i...,Montana
3,"[{'context': 'The phrase ""in whole or in part""...",Genocide
4,[{'context': 'The emergence of resistance of b...,Antibiotics


In [14]:
# df_squad_train['documents'].apply(lambda x : dict(eval(x)) )

In [15]:
df_squad_train.shape

(442, 2)

In [16]:
df_squad_train.iloc[0,0][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'questions': ['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
  'What is the Grotto at Notre Dame?',
  'What is in front of the Notre Dame Main Building?',
  'The Basilica of the Sacred heart at Notre Dame is beside to which structure?',
  'What sits on top of

In [17]:
df_squad_train.iloc[0,0][0]['context']

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [18]:
# df_squad_dev = pd.DataFrame(json_documents['squad_dev_doc.json'])
# df_dev = pd.DataFrame(json_documents['dev-v1.1.json'])
# df_train = pd.DataFrame(json_documents['train-v1.1.json'])

In [19]:
# df_squad_train.rename(columns={'passages':'documents'}, inplace=True)a

In [20]:
# len(df_squad_train.loc[0,'passages'])

In [21]:
# rearranging the dataframe such that it is easy to access (splitting from columns)
df_squad_train_new = df_squad_train.groupby('title').documents.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

In [22]:
# renaming the columns for understanding
df_squad_train_new.rename(columns={'questions':'question set'}, inplace=True)

In [23]:
df_squad_train_new.head()

,title,context,question set
0,2008_Sichuan_earthquake,The 2008 Sichuan earthquake or the Great Sichu...,"[What was the earthquake named?, What did the ..."
1,2008_Sichuan_earthquake,It is also known as the Wenchuan earthquake (C...,"[What was the focal depth of the quake?, How l..."
2,2008_Sichuan_earthquake,"Official figures (as of July 21, 2008 12:00 CS...","[How many were injured in Sichuan?, How many p..."
3,2008_Sichuan_earthquake,The earthquake had a magnitude of 8.0 Ms and 7...,"[What percentage of buildings were destroyed?,..."
4,2008_Sichuan_earthquake,According to a study by the China Earthquake A...,[What was the most displacement caused by the ...


In [24]:
df_squad_train_new.shape

(18896, 3)

In [25]:
# getting labels of documents, i.e. their names
doc_labels = df_squad_train_new.title.unique()

In [26]:
# Number of documents
len(doc_labels)

442

In [27]:
df_squad_train_new[df_squad_train_new['title']=='2008_Sichuan_earthquake'].context.sum()

'The 2008 Sichuan earthquake or the Great Sichuan earthquake, measured at 8.0 Ms and 7.9 Mw, and occurred at 02:28:01 PM China Standard Time at epicenter (06:28:01 UTC) on May 12 in Sichuan province, killed 69,197 people and left 18,222 missing.It is also known as the Wenchuan earthquake (Chinese: 汶川大地震; pinyin: Wènchuān dà dìzhèn; literally: "Great Wenchuan earthquake"), after the location of the earthquake\'s epicenter, Wenchuan County, Sichuan. The epicenter was 80 kilometres (50 mi) west-northwest of Chengdu, the provincial capital, with a focal depth of 19 km (12 mi). The earthquake was also felt in nearby countries and as far away as both Beijing and Shanghai—1,500 km (930 mi) and 1,700 km (1,060 mi) away—where office buildings swayed with the tremor. Strong aftershocks, some exceeding magnitude 6, continued to hit the area even months after the main quake, causing new casualties and damage.Official figures (as of July 21, 2008 12:00 CST) stated that 69,197 were confirmed dead, i

In [28]:
doc_data = []
for i in doc_labels:
    doc_data.append(df_squad_train_new[df_squad_train_new['title']==i].context.sum())

In [29]:
len(doc_data)

442

In [30]:
# DocIterator is a class which yields the labeled set of document separated by words
class DocIterator(object):
    
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
        
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield doc2vec.LabeledSentence(words=doc.split(),tags=[self.labels_list[idx]])

In [31]:
# getting the iterated documents and this iter documents consists of two simple lists(list of words and list of labels)
iter_docs = DocIterator(doc_data, doc_labels)

In [32]:
type(iter_docs)

__main__.DocIterator

In [29]:
model = doc2vec.Doc2Vec(documents=iter_docs,size=500, window=10, min_count=1, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate

In [30]:
model.docvecs.most_similar(['Dog'])

[('Predation', 0.9089356064796448),
 ('Mammal', 0.9032152891159058),
 ('Clothing', 0.8979605436325073),
 ('Sexual_orientation', 0.8824354410171509),
 ('Asthma', 0.8785061836242676),
 ('Nutrition', 0.8689378499984741),
 ('Bird', 0.8688399195671082),
 ('Immunology', 0.8676098585128784),
 ('Infection', 0.8650223016738892),
 ('Pain', 0.8639243245124817)]

In [31]:
query = 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [32]:
from nltk.tokenize import word_tokenize

In [33]:
test_words = word_tokenize(query)

In [34]:
new_doc_vec = model.infer_vector(test_words)

In [35]:
new_doc_vec

array([ -3.85934450e-02,  -4.19880003e-02,  -6.78244093e-03,
        -1.78211257e-02,  -2.06747893e-02,  -1.27542531e-02,
        -5.48308827e-02,   1.80494431e-02,  -2.86218654e-02,
         4.14801575e-02,   2.02486049e-02,   1.05772642e-02,
         9.61934961e-03,  -1.03572672e-02,   1.63555183e-02,
         1.12591553e-02,  -5.00448160e-02,   2.66569725e-04,
         1.11127393e-02,  -5.77856973e-02,   2.10598968e-02,
        -1.43226739e-02,   4.25462164e-02,   6.93476386e-03,
        -2.31826808e-02,   9.30795912e-04,  -2.19969433e-02,
         5.98635636e-02,   1.89549271e-02,   5.84820099e-02,
         2.28101537e-02,   1.70637481e-02,   4.51540900e-03,
        -1.52582433e-02,  -2.92836931e-02,   4.88779508e-02,
        -1.54226599e-02,  -6.79788589e-02,  -2.36718217e-03,
         2.93902550e-02,  -2.19621304e-02,  -1.39990766e-02,
         1.28445588e-02,  -8.92769545e-03,  -1.98267959e-02,
         1.84232146e-02,  -2.70263217e-02,  -1.31391436e-02,
        -2.00660359e-02,

In [36]:
model.docvecs.most_similar([new_doc_vec])

[('Yale_University', 0.47264623641967773),
 ('To_Kill_a_Mockingbird', 0.4688105583190918),
 ('Mary_(mother_of_Jesus)', 0.4543362259864807),
 ('Elizabeth_II', 0.43803682923316956),
 ('George_VI', 0.43785709142684937),
 ('Frédéric_Chopin', 0.41669750213623047),
 ('University_of_Notre_Dame', 0.40956512093544006),
 ('Eton_College', 0.40926796197891235),
 ('Carnival', 0.4067879617214203),
 ('Freemasonry', 0.4048232138156891)]

In [37]:
avg_sentence = np.zeros((500))
count = 0
for word in test_words:
    if word in model.wv.vocab:
        avg_sentence += model[word]
        count += 1
avg_sentence = avg_sentence / count

In [38]:
model.docvecs.most_similar([avg_sentence])

[('Pope_John_XXIII', 0.736782968044281),
 ('Edmund_Burke', 0.7187445759773254),
 ('Cardinal_(Catholicism)', 0.7158167958259583),
 ('Pope_Paul_VI', 0.713584303855896),
 ('Freemasonry', 0.7003039121627808),
 ('George_VI', 0.69888836145401),
 ('Baptists', 0.6941968202590942),
 ('England_national_football_team', 0.6846250891685486),
 ('Gregorian_calendar', 0.676995038986206),
 ('Athanasius_of_Alexandria', 0.6764561533927917)]